# QIR with Qiskit (via Q#)

## Classical instructions in circuits

### Run Qiskit with classical instructions
Qiskit has begun implementing some classical computation support as they expand their OpenQASM 3 support. These constructs, insofar as Qiskit can export them, can be consumed by Q#.

As an example, we can create a classical switch statement in Qiskit and run the program.

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit.circuit import (
    QuantumCircuit,
)

from qsharp import QSharpError, TargetProfile
from qsharp.interop.qiskit import QSharpBackend

qreg = QuantumRegister(3, name="q")
creg = ClassicalRegister(3, name="c")
qc = QuantumCircuit(qreg, creg)
qc.h([0, 1, 2])
qc.measure_all(add_bits=False)

with qc.switch(creg) as case:
    with case(7):
        qc.x(0)
    with case(1, 2):
        qc.z(1)
    with case(case.DEFAULT):
        qc.cx(0, 1)
qc.measure_all(add_bits=False)

backend = QSharpBackend()

print(backend.run(qc).result())

Using that same circuit, we can generate QIR which is used to run on quantum hardware.

In [ ]:
backend = QSharpBackend(target_profile=TargetProfile.Adaptive_RI)
print(backend.qir(qc))

Not all programs can run on all hardware. Here we can try to target the `Base` profile, but we will get detailed errors on which parts of the program aren't supported.

In [ ]:
try:
    backend.qir(qc, target_profile=TargetProfile.Base)
except QSharpError as e:
    print(e)

# QIR with OpenQASM (via Q#)

In [ ]:
import qsharp
from qsharp import init, TargetProfile, code, BitFlipNoise
from qsharp.qasm import import_qasm, run, compile

In [ ]:
source = """
    include "stdgates.inc";
    qubit q1;
    qubit q2;
    output int errors;
    for int i in [0:100] {
        h q1;
        cx q1, q2;
        bit[2] c;
        c[0] = measure q1;
        c[1] = measure q2;
        reset q1;
        reset q2;
        if (c[0] != c[1]) { errors += 1; }
    }
    """
result = run(source, shots=1,noise=BitFlipNoise(0.1))

print(result)

In [ ]:
init(target_profile=TargetProfile.Adaptive_RI)

result = import_qasm(
    """
    include "stdgates.inc";
    output int errors;
    qubit q;
    for int i in [0:100] {
        bit c = measure q;
        reset q;
        if (c != 0) { errors+=1; }
    }
    """,
    name="Program1",
)

In [ ]:
print(qsharp.compile(code.Program1))

In [ ]:
from pathlib import Path
source =  Path("qec.qasm").read_text()

qir = compile(source, target_profile=TargetProfile.Adaptive_RI)

print(qir)

In [ ]:
import qirrunner
from pathlib import Path

# Save the QIR to a file
source =  Path("qec.qasm").read_text()
qir_path = Path("qec.ll")
qir_path.write_text(str(qir))

# Run the QIR using qirrunner
qirrunner.run(str(qir_path), shots=10)